In [2]:
# import socket
# fimport socks
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import date
import time
from pandas import DataFrame
import os
import pandas as pd

In [3]:
from scraping.SportsBookOdds import get_odds
from utils.utils import transform_results_format, get_odds_dates

In [5]:
all_odds = get_odds_dates(['20200109'])

getting today's MoneyLine (1/6)
getting today's Spreads (2/6)
getting today's totals (3/6)
writing today's MoneyLine (1/6)
1/4
2/4
3/4
4/4
writing today's RunLine (2/6)
1/4
2/4
3/4
4/4
writing today's totals (3/6)
1/4
2/4
3/4
4/4


In [6]:
all_odds.head()

,key,date,ml_time,team,opp_team,ml_PIN,ml_FD,ml_HER,ml_BVD,ml_BOL,...,tot_PIN_line,tot_PIN_odds,tot_FD_line,tot_FD_odds,tot_HER_line,tot_HER_odds,tot_BVD_line,tot_BVD_odds,tot_BOL_line,tot_BOL_odds
0,20200109,21:28:15,away,Cleveland,Detroit,+259,+260,+250,+250,+255,...,220,+100,220,-102,220,-105,220,-110,220,-110
1,20200109,21:28:15,home,Detroit,Cleveland,-301,-290,-310,-300,-315,...,220,-110,220,-108,220,-105,220,-110,220,-110
2,20200109,21:28:15,away,Boston,Philadelphia,,+115,,,,...,,,214.5,-110,,,,,,
3,20200109,21:28:15,home,Philadelphia,Boston,,-135,,,,...,,,214.5,-110,,,,,,
4,20200109,21:28:15,away,Portland,Minnesota,,-145,,,,...,,,224.5,-110,,,,,,


In [8]:
from utils.utils import calc_agg_odds_prob

In [11]:
nbaOdds, _ = calc_agg_odds_prob(all_odds)

calculating odds for broker PIN
odds missing for broker PIN, skipping
calculating odds for broker HER
odds missing for broker HER, skipping
calculating odds for broker FD
calculating odds for broker BOL
odds missing for broker BOL, skipping
calculating odds for broker BVD
odds missing for broker BVD, skipping


In [13]:
nbaOdds.head()

,key,date,ml_time,team,opp_team,ml_PIN,ml_FD,ml_HER,ml_BVD,ml_BOL,...,tot_BOL_line,tot_BOL_odds,ml_FD_sign,ml_FD_dec,ml_FD_int,ml_FD_prob,ml_AGG_dec,ml_MAX_dec,ml_AGG_prob,ml_MAX_prob
0,20200109,21:28:15,away,Cleveland,Detroit,+259,+260,+250,+250,+255,...,220,-110,+,3.600000,260,0.277778,3.600000,3.600000,0.277778,0.277778
1,20200109,21:28:15,home,Detroit,Cleveland,-301,-290,-310,-300,-315,...,220,-110,-,1.344828,290,0.743590,1.344828,1.344828,0.743590,0.743590
2,20200109,21:28:15,away,Boston,Philadelphia,,+115,,,,...,,,+,2.150000,115,0.465116,2.150000,2.150000,0.465116,0.465116
3,20200109,21:28:15,home,Philadelphia,Boston,,-135,,,,...,,,-,1.740741,135,0.574468,1.740741,1.740741,0.574468,0.574468
4,20200109,21:28:15,away,Portland,Minnesota,,-145,,,,...,,,-,1.689655,145,0.591837,1.689655,1.689655,0.591837,0.591837


In [16]:
cols = ['key', 'ml_time', 'team', 'ml_AGG_dec', 'ml_AGG_prob']
nbaOdds[cols]

,key,ml_time,team,ml_AGG_dec,ml_AGG_prob
0,20200109,away,Cleveland,3.600000,0.277778
1,20200109,home,Detroit,1.344828,0.743590
2,20200109,away,Boston,2.150000,0.465116
3,20200109,home,Philadelphia,1.740741,0.574468
4,20200109,away,Portland,1.689655,0.591837
5,20200109,home,Minnesota,2.250000,0.444444
6,20200109,away,Houston,1.689655,0.591837
7,20200109,home,Oklahoma City,2.250000,0.444444


In [17]:
nbaOdds.loc[nbaOdds.ml_AGG_prob > 0.65, cols]

,key,ml_time,team,ml_AGG_dec,ml_AGG_prob
1,20200109,home,Detroit,1.344828,0.74359


In [15]:
betsPlaced = nbaResultsOdds.loc[nbaResultsOdds.ml_AGG_prob > 0.65, cols]
exp = 22
total_wagered = (betsPlaced.ml_AGG_dec ** exp).sum()
total_return = (betsPlaced.loc[betsPlaced.WinLoss].ml_AGG_dec ** (1+exp)).sum()
precent_return = (total_return -  total_wagered) / total_wagered

betsPlaced['wager'] = betsPlaced['ml_AGG_dec'] ** exp
betsPlaced['return'] = betsPlaced.ml_AGG_dec ** (1+exp) * betsPlaced.WinLoss

betsPlaced['PnL'] = betsPlaced['return'] - betsPlaced['wager']

print('total wagered {}'.format(total_wagered))
print('total return {}'.format(total_return))
print('profit {}'.format(total_return - total_wagered))
print('% return {}'.format(precent_return))
print('max bet ${}'.format((betsPlaced.ml_AGG_dec ** exp).max()))
print('% winning days {}'.format((betsPlaced.groupby('key')['PnL'].sum() > 0).mean()))


print('\nwager summary:')
print(betsPlaced.groupby('key')['wager'].sum().describe())

print('\ndaily return summary:')
print((betsPlaced.groupby('key')['return'].sum() - betsPlaced.groupby('key')['wager'].sum()).describe())



total wagered 456226.33012129355
total return 519678.759766196
profit 63452.42964490247
% return 0.13908103381063702
max bet $12633.339622037116
% winning days 0.7540983606557377

wager summary:
count       61.000000
mean      7479.120166
std       8126.267498
min          7.588401
25%       1840.163585
50%       4099.094922
75%      10084.507604
max      32155.526229
Name: wager, dtype: float64

daily return summary:
count       61.000000
mean      1040.203765
std       4553.492646
min     -10610.488426
25%          0.732246
50%        471.346939
75%       2210.333414
max      15562.033193
dtype: float64


In [16]:
betsPlaced['PnL'] = betsPlaced['return'] - betsPlaced['wager']

In [17]:
(betsPlaced.groupby('key')['PnL'].sum() > 0).mean()

0.7540983606557377